In [1]:
import pandas as pd
import numpy as np
from glob import glob
import os

# fileName - iterate through entire folder :)
#fileName = '3S3-1_Crestwood_Feeder_Details.xlsx'

#input directory
inputDirectory = 'Metsco_Feeder_Reports'

# define filepath and sort the file list
filesList = glob(os.path.join(inputDirectory, '*.xlsx'))
numFiles = len(filesList)
print('Number of Feeders: ', numFiles)
sortedFileList = sorted(filesList)

# variables
dictFeeders = {}
allNodes_list = pd.DataFrame()

# read text files in tweet_input directory
for f in sortedFileList:

    fileName = os.path.basename(f).split('_')
    FeederKey = fileName[0]
    #print(FeederKey)
    
    if ('$' not in FeederKey):
        # Read CYME Feeder xlsx file into dataframes
        with pd.ExcelFile(f) as xlsx:
            #dfTopology = pd.read_excel(xlsx, 'Topology', index_col=None, na_values=['NA']) # IGNORE for now
            dfTopology = pd.read_excel(xlsx, 'Topology') # 280 rows
            dfSpotLoads = pd.read_excel(xlsx, 'Spot Loads') # Tot:239 - R/Y/B: 116/108/103 values; based on phases
            dfLoads = pd.read_excel(xlsx, 'Loads') # 239 rows; 'Spot Number\n' col contains unique tx ids
            dfCables = pd.read_excel(xlsx, 'Cables')
            dfSwitches = pd.read_excel(xlsx, 'Switches') # 41 items
            dfNodes = pd.read_excel(xlsx, 'Nodes') # 249 items
            dfOHlines = pd.read_excel(xlsx, 'OverheadLinesByPhase') #Neutral - 94, Section Id - 381
            dfFuses = pd.read_excel(xlsx, 'Fuses') # 44 items

            # # Strip '\n' from column headers
            dfTopology.rename(columns=lambda x: x.replace('\n',''), inplace=True)
            dfSpotLoads.rename(columns=lambda x: x.replace('\n',''), inplace=True)
            dfLoads.rename(columns=lambda x: x.replace('\n',''), inplace=True)
            dfCables.rename(columns=lambda x: x.replace('\n',''), inplace=True)
            dfSwitches.rename(columns=lambda x: x.replace('\n',''), inplace=True)
            dfNodes.rename(columns=lambda x: x.replace('\n',''), inplace=True)
            dfOHlines.rename(columns=lambda x: x.replace('\n',''), inplace=True)
            dfFuses.rename(columns=lambda x: x.replace('\n',''), inplace=True)
            #print(dfNodes.columns)
            allNodes_list = allNodes_list.append(dfNodes)

            # Split 'Nodes_Node Id' to 'NodeID_1' and 'NodeID_2' for 'SwitchRegion'
            #dfSwitches['NodeID_1'], dfNodesCopy['NodeID_2'] = zip(*dfNodesCopy['Nodes_Node Id'].
            #apply(lambda x: x.split('_') if '_' in x else (x, np.nan)))
            
#             cableLength = sum(dfCables['Length(m)'])/1000
#             txCounts = len(dfLoads['Spot Number'])
#             fuseCounts = len(dfFuses['Equipment Id'])
#             switchCounts = len(dfSwitches['Equipment Id'])
            
            
            #print(FeederKey,':',cableLength,'kms')
            #dictFeeders.update({FeederKey:[cableLength, txCounts, switchCounts,fuseCounts]})
        
# Write to excel file
# dfFdrs = pd.DataFrame(dictFeeders)
# dfFdrs = dfFdrs.transpose()
# dfFdrs.columns =['Cables','Transformer', 'Switches','Fuses']
#print(dfFdrs)
# print(ugSwitches)
# MasterFile = pd.ExcelWriter('AllFeeders.xlsx')
# dfFdrs.to_excel(MasterFile, 'Sheet1')
# MasterFile.save()

MasterFile = pd.ExcelWriter('AllNodes.xlsx')
allNodes_list.to_excel(MasterFile, 'Sheet1')
MasterFile.save()

print(allNodes_list.shape)

Number of Feeders:  36
(12096, 3)


In [87]:
# fileName - iterate through entire folder :)
fileName = 'Original_FiveAssetClasses_Copy.xlsx'
#fileNameOtherDevices = 'Other Device Numbers.xls'
#fileNamePolesClassHeight = 'Poles_class_height.xlsx' # Poles_class_height table

# Read xlsx file into dataframes
with pd.ExcelFile(fileName) as xlsx:
    #dfTopology = pd.read_excel(xlsx, 'Topology', index_col=None, na_values=['NA']) # IGNORE for now
    dfTransformersV1 = pd.read_excel(xlsx, 'Transformers') # 280 rows
    dfSwitchesV1 = pd.read_excel(xlsx, 'Switches') # Tot:239 - R/Y/B: 116/108/103 values; based on phases
    dfPolesV1 = pd.read_excel(xlsx, 'Poles') # 239 rows; 'Spot Number\n' col contains unique tx ids
    dfCablesV1 = pd.read_excel(xlsx, 'UGPrimaryCables')
    dfFusesV1 = pd.read_excel(xlsx, 'Fuses') # 44 items
    dfUGStructuresV1 = pd.read_excel(xlsx,'UGStructures')

    
# df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Remove ' ' in 1R 1-S but some FEEDERID values are float. Will do it manually
# dfTransformersV1['FEEDERID'] = dfTransformers['FEEDERID'].apply(lambda x: x.replace(' ', '') if ' ' in x else x)
# dfSwitchesV1['FEEDERID'] = dfSwitchesV1['FEEDERID'].apply(lambda x: x.replace(' ', '') if ' ' in x else x)
# dfCablesV1['FEEDERID'] = dfCablesV1['FEEDERID'].apply(lambda x: x.replace(' ', '') if ' ' in x else x)
# dfFusesV1['FEEDERID'] = dfFusesV1['FEEDERID'].apply(lambda x: x.replace(' ', '') if ' ' in x else x)

Summary = {'Transformers:': dfTransformersV1.shape, 'Switches:': dfSwitchesV1.shape,'Poles:': dfPolesV1.shape, 
           'Cables:': dfCablesV1.shape, 'Fuses:':dfFusesV1.shape, 'UGStructures:':dfUGStructuresV1.shape}
dfSummary = pd.DataFrame(Summary)

# Make one copy
dfTransformersV2 = dfTransformersV1
dfSwitchesV2 = dfSwitchesV1
dfPolesV2 = dfPolesV1
dfCablesV2 = dfCablesV1
dfFusesV2 = dfFusesV1
dfUGStructuresV2 = dfUGStructuresV1

# 17 columns dropped
dropCommonColumns = ['OBJECTID','WORKORDERID','FIELDVERIFY','COMMENTS','CREATIONUSER','DATECREATED','LASTUSER',
                     'DATEMODIFIED','WORKREQUESTID','DESIGNID','WORKLOCATIONID','WMSID','WORKFLOWSTATUS',
                     'WORKFUNCTION','GISONUMBER','GISOTYPENBR','OWNERSHIP']

#*************************#
#****DEFINITIONS**********#
#*************************#
def drop_columns(dfAssetClass, dropColumns):
    dfAssetClass = dfAssetClass.drop(dropColumns, axis=1)
    return dfAssetClass

def new_columns(dfAssetClass, numAssetRows, columnID):
    dfAssetClass[columnID] = pd.DataFrame(np.empty([numAssetRows,1]).cumsum(axis=1))
    dfAssetClass.loc[:,columnID] = np.nan
    return dfAssetClass[columnID]

#*************************#
#****DEFINITIONS**********#
#*************************#

#Drop all common columns 
dfSwitchesV2 = drop_columns(dfSwitchesV2, dropCommonColumns)
dfTransformersV2 = drop_columns(dfTransformersV2, dropCommonColumns)
dfFusesV2 = drop_columns(dfFusesV2,dropCommonColumns)
dfCablesV2 = drop_columns(dfCablesV2, dropCommonColumns)
dfUGStructuresV2 = drop_columns(dfUGStructuresV2, dropCommonColumns)
dfPolesV2 = drop_columns(dfPolesV2, dropCommonColumns)

# Make one copy
dfTransformers = dfTransformersV2
dfSwitches = dfSwitchesV2
dfPoles = dfPolesV2
dfCables = dfCablesV2
dfFuses = dfFusesV2
dfUGStructures = dfUGStructuresV2

# SummaryV2 = {'Transformers:': dfTransformers.shape, 'Switches:': dfSwitches.shape,'Poles:': dfPoles.shape, 
#            'Cables:': dfCables.shape, 'Fuses:':dfFuses.shape, 'UGStructures:':dfUGStructures.shape}
# dfSummaryV2 = pd.DataFrame(SummaryV2)

In [109]:
# numTxRows = len(dfTransformers['DEVICENUMBER'])
# numPolesRows = len(dfPoles['DEVICENUMBER'])
# numSwitchRows = len(dfSwitches['DEVICENUMBER'])
# numFusesRows = len(dfFuses['DEVICENUMBER'])

# df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
dfGroupedTxFdrs = pd.DataFrame(dfTransformers.groupby(by=['FEEDERID'])['DEVICENUMBER'].count())
dfGroupedSwitchFdrs = pd.DataFrame(dfSwitches.groupby(by=['FEEDERID'])['DEVICENUMBER'].count())
dfGroupedFusesFdrs = pd.DataFrame(dfFuses.groupby(by=['FEEDERID'])['DEVICENUMBER'].count())
dfGroupedCableFdrs = pd.DataFrame(dfCables.groupby(by=['FEEDERID'])['MEASUREDLENGTH'].sum())

# dfGroupedTxFdrs['FdrID']= dfGroupedTxFdrs.index
# dfGroupedSwitchFdrs['FdrID']= dfGroupedSwitchFdrs.index
# dfGroupedFusesFdrs['FdrID']= dfGroupedFusesFdrs.index
# dfGroupedCableFdrs['FdrID']= dfGroupedCableFdrs.index

# cols = ['FdrID', 'DEVICENUMBER']
# cols2 = ['FdrID', 'MEASUREDLENGTH']
# dfGroupedTxFdrs = dfGroupedTxFdrs[cols]
# dfGroupedSwitchFdrs = dfGroupedSwitchFdrs[cols]
# dfGroupedFusesFdrs = dfGroupedFusesFdrs[cols]
# dfGroupedCableFdrs = dfGroupedCableFdrs[cols2]

# frames = [dfGroupedCableFdrs, dfGroupedTxFdrs, dfGroupedSwitchFdrs, dfGroupedFusesFdrs]
# dfGroupedAll = pd.concat(frames, ignore_index=True, keys='FdrID')
# print(dfGroupedAll.head(2))

#print(dfGroupedTxFdrs.head(3))
#print(dfGroupedSwitchFdrs.head(3))
# print(dfGroupedFusesFdrs.head(3))
#print(len(dfGroupedTxFdrs))
# print(len(dfGroupedSwitchFdrs))
# print(len(dfGroupedFusesFdrs))


frames = [dfGroupedCableFdrs, dfGroupedTxFdrs, dfGroupedSwitchFdrs, dfGroupedFusesFdrs]
# dfGroupedAll = dfGroupedCableFdrs.append(dfGroupedTxFdrs)

# print(dfGroupedAll.head(2))
#dfGroupedAll = pd.DataFrame(dfGroupedTxFdrs.merge(dfGroupedSwitchFdrs, how='inner', on='FdrID'))

#pd.concat([s3, s4, s5], axis=1, keys=['red','blue','yellow'])
#dfGroupedAll = pd.concat(frames, ignore_index=True, keys=['FdrID', 'Cables', 'Tx', 'Switches'])
dfGroupedAll = dfGroupedCableFdrs.append([dfGroupedTxFdrs, dfGroupedSwitchFdrs, dfGroupedFusesFdrs])
# print(dfGroupedAll.head(2))
#dfGroupedAll = dfGroupedAll.merge(dfGroupedFusesFdrs, how='left', on='FdrID')
#print(dfGroupedAll.head(2))
MasterFile = pd.ExcelWriter('AllFeedersGIS.xlsx')
dfGroupedAll.to_excel(MasterFile, 'GIS')
MasterFile.save()

In [96]:
#seriesFdrs = pd.DataFrame(dictFeeders, index=['Cable (kms)', 'Tx Counts', 'Switch Counts', 'Fuse Counts'])
#print(seriesFdrs)
#dfFdrs = pd.DataFrame(dictFeeders.items(), columns=['Feeder', 'Cable (kms)', 'Tx Counts', 'Switch Counts', 'Fuse Counts'])
#dfFdrs = pd.DataFrame.to_dict(dictFeeders, orient='index')
dfFdrs = pd.DataFrame(dictFeeders)
dfFdrs = dfFdrs.transpose()
dfFdrs.columns =['Cables','Transformer', 'Switches','Fuses']
# print(dfFdrs)

In [21]:
# Quick and fast peek into sheet names
#xls = xlrd.open_workbook(r'<path_to_your_excel_file>', on_demand=True)
# Fdr1 = xlrd.open_workbook(r'3S3-1_Crestwood_Feeder_Details.xlsx', on_demand=True)
# print(Fdr1.sheet_names()) # <- remember: xlrd sheet_names is a function, not a property

#import timeit
#timeit.timeit('pd.read_excel(fileName, "Topology")', number=10000)
#%timeit(pd.read_excel(fileName, 'Topology')) # 561 ms per loop (best out of 3)
#%timeit(pd.ExcelFile(fileName).parse('Topology')) # 539 ms per loop (best out of 3)


# Sources:
# http://www.swegler.com/becky/blog/2014/08/06/useful-pandas-snippets/
# http://stackoverflow.com/questions/12250024/how-to-obtain-sheet-names-from-xls-files-without-loading-the-whole-file-in-pytho
# Read excel sheets: http://pandas.pydata.org/pandas-docs/stable/io.html#io-excel-reader

#print(dfCables.count())
#Feeder = pd.read_excel('3S3-1_Crestwood_Feeder_Details.xlsx', 0, index_col=None, na_values=['NA'])
#print(Feeder)

#timeit.timeit('"-".join(str(n) for n in range(100))', number=10000)

#import timeit
#timeit.timeit('pd.read_excel(fileName, "Topology")', number=10000)
#%timeit(pd.read_excel(fileName, 'Topology')) # 561 ms per loop (best out of 3)
#%timeit(pd.ExcelFile(fileName).parse('Topology')) # 539 ms per loop (best out of 3)

#print('Sum of cable length:', sum(dfCables['Length(m)'])/1000, 'kms')
cableLength = sum(dfCables['Length(m)'])/1000
dictMaster = {'Sum:' : cableLength, 'total: ':cableLength}
dfMaster = pd.Series(dictMaster, )
print(dfMaster)
#pd.DataFrame(d.items(), columns=['Date', 'DateValue'])
# MasterFile = pd.ExcelWriter('master.xlsx')
# dfMaster.to_excel(MasterFile, 'Sheet1')
# MasterFile.save()

Sum:       24.1101
total:     24.1101
dtype: float64


In [4]:

# ****************************
# A. NODES sheet
# ****************************
# 1. Split 'Node Id' to 'NodeID_1' and 'NodeID_2'
dfNodes['NodeID_1'], dfNodes['NodeID_2'] = zip(*dfNodes['Node Id'].apply(lambda x: x.split('_') if '_' in x else (x, np.nan)))
# 2. Create a 'Copy' dataframe
dfNodesCopy = pd.DataFrame(dfNodes)
# 3. Rename al column headers to 'Nodes_' + x
dfNodesCopy.rename(columns=lambda x: 'Nodes_'+x, inplace=True)
#print(dfNodes_Copy.count())

# ****************************
# B. MASTER SPREADSHEET
# ****************************
# Copy dfNodesCopy into dfMaster
dfMaster = pd.DataFrame(dfNodesCopy)
# print(dfMaster.count())
# Nodes_NodeID_1 and Nodes_NodeID_2 are keys

# ****************************
# C. Topology sheet
# ****************************
# 1. No renaming here,so freate a 'copy' dataframe
dfTopologyCopy = pd.DataFrame(dfTopology)
# 2. Rename all column headers to 'Topology_' + x
dfTopologyCopy.rename(columns=lambda x: 'Topology_'+x, inplace=True)
#print(dfTopologyCopy.count())

# 3. Combine topology sheet
# pd.merge(frame_1, frame_2, left_on = 'county_ID', right_on = 'countyid')
# dfFinal = 
# Topology - more match with 'Topology_Coord. Y' over 'Topology_Coord. X'
dfMaster = pd.merge(dfMaster, dfTopologyCopy, how='outer', left_on='Nodes_NodeID_2', right_on ='Topology_Coord. Y')
#print(dfMaster.count())


# ****************************
# D. Fuses sheet 
# ****************************
# 1. Split 'From Node' to 'FromNode_xCoord' and 'FromNode_yCoord'
dfFuses['FromNode_xCoord'], dfFuses['FromNode_yCoord'] = zip(*dfFuses['From Node'].apply(lambda x: x.split('_') if '_' in x else (x, np.nan)))
# 2. Split 'To Node' to 'ToNode_FuseID' and 'ToNode_FdrID'
dfFuses['ToNode_FuseID'], dfFuses['ToNode_FdrID'] = zip(*dfFuses['To Node'].apply(lambda x: x.split('_') if '_' in x else (x, np.nan)))
# 3. Create a 'Copy' dataframe
dfFusesCopy = pd.DataFrame(dfFuses)
# 4. Rename all column headers to 'Fuses_' + x
dfFusesCopy.rename(columns=lambda x:'Fuses_'+x, inplace=True)
# 5. Combine Fuses sheet with Master
dfMaster = pd.merge(dfMaster, dfFusesCopy, how='outer', left_on='Nodes_NodeID_1', right_on ='Fuses_FromNode_xCoord')
#print(dfMaster.count())

# ****************************
# D1. Output excel file - For VERIFICATION purposes
# ****************************
# Verify the excel file 
# http://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.to_excel.html
# http://stackoverflow.com/questions/29974672/writing-pandas-dataframe-to-excel-with-different-formats-for-different-columns
# MasterFile = pd.ExcelWriter('master.xlsx')
# dfMaster.to_excel(MasterFile, 'Sheet1')
# MasterFile.save()


In [7]:

# ****************************
# E. Switch sheet 
# ****************************
# 1. Split 'From Node' to 'FromNode_1' and 'FromNode_2'
dfSwitches['FromNode_1'], dfSwitches['FromNode_2'] = zip(*dfSwitches['From Node'].apply(lambda x: x.split('_') if '_' in x else (x, np.nan)))
# 2. Create a 'Copy' dataframe
dfSwitchesCopy = pd.DataFrame(dfSwitches)
# 3. Rename all column headers to 'Switches_' + x
dfSwitchesCopy.rename(columns=lambda x:'Switches_'+x, inplace=True)
# 4. Combine Switches sheet with Master: 
# 4.1 First with 'Switches_FromNode_1' - NodeID_1 also has '109-D'/'7-S' switch id :)
dfMaster = pd.merge(dfMaster, dfSwitchesCopy, how='outer', left_on='Nodes_NodeID_1', right_on ='Switches_FromNode_1')
# 4.2 Second with 'Section Id' of FusesCopy - maybe not necessary

# ****************************
# F. Transformer aka "Loads" in CYME
# ****************************
# 
# 1. Split 'From Node' to 'FromNode_1' and 'FromNode_2'
dfLoads['FromNode_1'], dfLoads['FromNode_2'] = zip(*dfLoads['From Node'].apply(lambda x: x.split('_') if '_' in x else (x, np.nan)))
# 2. Create a 'Copy' dataframe
dfLoadsCopy = pd.DataFrame(dfLoads)
# 3. Rename all column headers to Loads_' + x
dfLoadsCopy.rename(columns=lambda x:'Loads_'+x, inplace=True)
# 4. Combine all Loads with 'FromNode_1'  with dfMaster
dfMaster = pd.merge(dfMaster, dfLoadsCopy, how='outer', left_on='Nodes_NodeID_1', right_on ='Loads_FromNode_1')
# 4.2 May need to combine dfLoadsCopy with dfSpotLoads if tx nameplate rating not same

# ****************************
# D1. Output excel file
# ****************************
# Verify the excel file 
# http://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.to_excel.html
# http://stackoverflow.com/questions/29974672/writing-pandas-dataframe-to-excel-with-different-formats-for-different-columns
MasterFile = pd.ExcelWriter('master.xlsx')
dfMaster.to_excel(MasterFile, 'Sheet1')
MasterFile.save()
# ****************************
# G. PRID to each region
# ****************************
# combine PRID to Tx?

# ****************************
# H. Cable 
# ****************************
# combine cable and conductors

# ****************************
# I. Conductors  
# ****************************
# combine poles

# ****************************
# J. Poles 
# ****************************
# Output excel file

# Delete dataframes or drop columns
#dfNodes_Copy = dfNodes_Copy.drop('Nodes_Network Id\n',1)
# del dfNodes_Copy

#print(dfMaster)

#dfFuses['FromNode_xCoord'], dfFuses['FromNode_yCoord'] = zip(*dfFuses['From Node\n'].apply(lambda x: x.split('_') if '_' in x else (x, np.nan)))
#dfFuses['ToNode_FuseID'], dfFuses['ToNode_FdrID'] = zip(*dfFuses['To Node\n'].apply(lambda x: x.split('_') if '_' in x else (x, np.nan)))

#dfMaster['FromNode_xCoord'], dfMaster['FromNode_yCoord'] = zip(*dfFuses['From Node\n'].apply(lambda x: x.split('_') if '_' in x else (x, np.nan)))
#dfMaster['ToNode_FuseID'], dfMaster['ToNode_FdrID'] = zip(*dfFuses['To Node\n'].apply(lambda x: x.split('_') if '_' in x else (x, np.nan)))
#print(dfMaster['FromNode_xCoord'].count()) # 44 values :)
#print(dfMaster['FromNode_xCoord'])

# ['Cmd' + '/' to uncomment]
# *****Rename all columns for each sheets 
# Rename the _copy versions
# dfTopologyCopy.rename(columns=lambda x: 'Topology_'+x, inplace=True)
# dfSpotLoadsCopy.rename(columns=lambda x: 'SpotLoads_'+x, inplace=True)
# dfLoadsCopy.rename(columns=lambda x: 'Loads_'+x, inplace=True)
# dfCablesCopy.rename(columns=lambda x: 'Cables_'+x, inplace=True)
# dfSwitchesCopy.rename(columns=lambda x: 'Switches_'+x, inplace=True)
# dfNodesCopy.rename(columns=lambda x: 'Nodes_'+x, inplace=True)
# dfOHlinesCopy.rename(columns=lambda x: 'OHlinesByPhase_'+x, inplace=True)
# dfFusesCopy.rename(columns=lambda x: 'Fuses_'+x, inplace=True)
# ******rename the original version****
# dfTopology.rename(columns=lambda x: 'Topology_'+x, inplace=True)
# dfSpotLoads.rename(columns=lambda x: 'SpotLoads_'+x, inplace=True)
# dfLoads.rename(columns=lambda x: 'Loads_'+x, inplace=True)
# dfCables.rename(columns=lambda x: 'Cables_'+x, inplace=True)
# dfSwitches.rename(columns=lambda x: 'Switches_'+x, inplace=True)
# dfNodes.rename(columns=lambda x: 'Nodes_'+x, inplace=True)
# dfOHlines.rename(columns=lambda x: 'OHlinesByPhase_'+x, inplace=True)
# dfFuses.rename(columns=lambda x: 'Fuses_'+x, inplace=True)

